Dataset Source: https://www.kaggle.com/freddejn/flights

### 1- Load the data

In [228]:
!pip install pyspark

In [229]:
from pyspark.sql import SparkSession

In [230]:
spark = SparkSession.builder.getOrCreate()

In [231]:
import os
MAIN_DIRECTORY = os.getcwd()

In [232]:
file_path = MAIN_DIRECTORY + '/data/flights.csv'

In [328]:
df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(file_path)

In [275]:
df.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [235]:
df.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

In [236]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [237]:
#to create temp. view for spark sql
df.createOrReplaceTempView('flights')

In [238]:
spark.sql('SELECT * FROM flights LIMIT 5').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

### 2- Statistical descriptions of every airlines

In [239]:
df_describe = df.describe()

In [240]:
df_describe.show()

+-------+--------------------+------------------+------------------+-----------------+-------+------------------+-----------+--------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+--------------------+--------------------+-------------------+------------------+--------------------+------------------+-------------------+------------------+
|summary|                YEAR|             MONTH|               DAY|      DAY_OF_WEEK|AIRLINE|     FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|    DEPARTURE_TIME|   DEPARTURE_DELAY|          TAXI_OUT|        WHEELS_OFF|    SCHEDULED_TIME|      ELAPSED_TIME|          AIR_TIME|         DISTANCE|         WHEELS_ON|          TAXI_IN| SCHEDULED_ARRIVAL|      ARRIVAL_TIME|    ARRIV

In [241]:
#to only describe 'YEAR' column
df.describe('YEAR').show()

+-------+--------------------+
|summary|                YEAR|
+-------+--------------------+
|  count|             1103779|
|   mean|              2015.0|
| stddev|1.136077604323535...|
|    min|                2015|
|    max|                2015|
+-------+--------------------+



### 3- How many number of flights does each airline company has?

In [242]:
#DataFrame API
df_count = df.groupBy(df['AIRLINE']).count().orderBy(df['AIRLINE'].desc()).show()

+-------+------+
|AIRLINE| count|
+-------+------+
|     WN|233719|
|     VX| 10932|
|     US| 77709|
|     UA| 92474|
|     OO|112495|
|     NK| 20583|
|     MQ| 68589|
|     HA| 14810|
|     F9| 15405|
|     EV|117285|
|     DL|155609|
|     B6| 50557|
|     AS| 31138|
|     AA|102474|
+-------+------+



In [243]:
#Spark SQL
df_count = spark.sql('SELECT AIRLINE, COUNT(TAIL_NUMBER) AS Count FROM flights GROUP BY AIRLINE ORDER BY Count DESC')
df_count.show()

+-------+------+
|AIRLINE| Count|
+-------+------+
|     WN|232696|
|     DL|155583|
|     EV|117231|
|     OO|112471|
|     AA|101513|
|     UA| 90128|
|     US| 74641|
|     MQ| 68541|
|     B6| 50557|
|     AS| 31138|
|     NK| 20583|
|     F9| 15181|
|     HA| 14780|
|     VX| 10932|
+-------+------+



### 4- What were the total flights that each airplane has made?

In [244]:
#DataFrame API
df_flights = df.groupBy(df['TAIL_NUMBER']).count().orderBy(df['TAIL_NUMBER'].desc()).show()

+-----------+-----+
|TAIL_NUMBER|count|
+-----------+-----+
|     N9EAMQ|  356|
|     N999DN|  290|
|     N998DL|  323|
|     N998AT|  380|
|     N997DL|  313|
|     N996DL|  322|
|     N995DL|  279|
|     N995AT|  360|
|     N994DL|  281|
|     N994AT|  369|
|     N993DL|  293|
|     N993AT|   65|
|     N992DL|  291|
|     N991DL|  299|
|     N991AT|  386|
|     N990DL|  278|
|     N990AT|  392|
|     N989DL|  273|
|     N989AT|  381|
|     N988DL|  306|
+-----------+-----+
only showing top 20 rows



In [245]:
#Spark SQL
df_flights = spark.sql('SELECT TAIL_NUMBER, count(*) as Count FROM Flights GROUP BY TAIL_NUMBER ORDER BY Count DESC')
df_flights.show()

+-----------+-----+
|TAIL_NUMBER|Count|
+-----------+-----+
|       null| 7804|
|     N477HA|  854|
|     N488HA|  816|
|     N476HA|  811|
|     N485HA|  807|
|     N486HA|  783|
|     N484HA|  779|
|     N493HA|  766|
|     N492HA|  750|
|     N489HA|  748|
|     N480HA|  735|
|     N487HA|  714|
|     N491HA|  708|
|     N483HA|  694|
|     N481HA|  616|
|     N528SW|  557|
|     N511SW|  555|
|     N523SW|  554|
|     N526SW|  549|
|     N478HA|  530|
+-----------+-----+
only showing top 20 rows



### 5- What was the most preferred airline?

In [246]:
#DataFrame API
df.groupBy(df['AIRLINE']).count().orderBy(df['AIRLINE'].desc()).head()[0:2]

('WN', 233719)

In [247]:
#Spark SQL
df_count = spark.sql('SELECT AIRLINE, COUNT(TAIL_NUMBER) AS Count FROM flights GROUP BY AIRLINE ORDER BY Count DESC LIMIT 1')
df_count.show()

+-------+------+
|AIRLINE| Count|
+-------+------+
|     WN|232696|
+-------+------+



### 6- Delays: arrival or departure?

In [248]:
#DataFrame API
df_arrivaldelay = df.groupby('AIRLINE').agg({'ARRIVAL_DELAY':'mean'}).show()

+-------+-------------------+
|AIRLINE| avg(ARRIVAL_DELAY)|
+-------+-------------------+
|     UA|   6.91378734954502|
|     NK| 14.464639426666004|
|     AA|  7.999651385741677|
|     EV|  10.13289192807265|
|     B6| 13.147467465789308|
|     DL| 2.5077849338247833|
|     OO|  9.528759967005774|
|     F9| 23.754571258828964|
|     US|   5.25674294780434|
|     MQ| 19.792931290163125|
|     HA|   4.45045045045045|
|     AS|-0.5548200785229891|
|     VX|  5.121751412429378|
|     WN| 3.5873994957678943|
+-------+-------------------+



In [249]:
#SPARK SQL
df_arrivaldelay = spark.sql('SELECT AIRLINE, round(avg(ARRIVAL_DELAY), 2) AS Average_Arrival_Delay FROM flights GROUP BY AIRLINE ORDER BY Average_Arrival_Delay')
df_arrivaldelay.show()

+-------+---------------------+
|AIRLINE|Average_Arrival_Delay|
+-------+---------------------+
|     AS|                -0.55|
|     DL|                 2.51|
|     WN|                 3.59|
|     HA|                 4.45|
|     VX|                 5.12|
|     US|                 5.26|
|     UA|                 6.91|
|     AA|                  8.0|
|     OO|                 9.53|
|     EV|                10.13|
|     B6|                13.15|
|     NK|                14.46|
|     MQ|                19.79|
|     F9|                23.75|
+-------+---------------------+



In [250]:
#DataFrame API
df_departuredelay = df.groupby('AIRLINE').agg({'DEPARTURE_DELAY':'mean'}).show()

+-------+--------------------+
|AIRLINE|avg(DEPARTURE_DELAY)|
+-------+--------------------+
|     UA|  14.651451269305877|
|     NK|   15.69885263001043|
|     AA|  11.153218059558117|
|     EV|   11.17374513898891|
|     B6|  15.329182311222361|
|     DL|    9.41277289147165|
|     OO|  11.009628272108099|
|     F9|  22.735327053553785|
|     US|   7.305864069345266|
|     MQ|   17.71331684222597|
|     HA|  1.5524357239512856|
|     AS|   2.948009564430658|
|     VX|  10.139761748428853|
|     WN|   9.840593431563372|
+-------+--------------------+



In [251]:
#SPARK SQL
df_departuredelay = spark.sql('SELECT AIRLINE, round(avg(DEPARTURE_DELAY), 2) AS Average_Departure_Delay FROM flights GROUP BY AIRLINE ORDER BY Average_Departure_Delay')
df_departuredelay.show()

+-------+-----------------------+
|AIRLINE|Average_Departure_Delay|
+-------+-----------------------+
|     HA|                   1.55|
|     AS|                   2.95|
|     US|                   7.31|
|     DL|                   9.41|
|     WN|                   9.84|
|     VX|                  10.14|
|     OO|                  11.01|
|     AA|                  11.15|
|     EV|                  11.17|
|     UA|                  14.65|
|     B6|                  15.33|
|     NK|                   15.7|
|     MQ|                  17.71|
|     F9|                  22.74|
+-------+-----------------------+



Therefore, most delays occured during departure.

### 7- How many cancelled flights?

In [329]:
#DataFrame API
df = df.filter(df['CANCELLED'] == 1).groupby('AIRLINE', 'TAIL_NUMBER', 'CANCELLED').sum('CANCELLED').show()

+-------+-----------+---------+--------------+
|AIRLINE|TAIL_NUMBER|CANCELLED|sum(CANCELLED)|
+-------+-----------+---------+--------------+
|     MQ|     N674MQ|        1|            35|
|     OO|     N756SK|        1|            12|
|     OO|     N773SK|        1|             6|
|     AA|     N3GEAA|        1|             7|
|     B6|     N929JB|        1|             5|
|     DL|     N905DE|        1|             9|
|     DL|     N302NB|        1|             4|
|     NK|     N635NK|        1|            11|
|     EV|     N176PQ|        1|             6|
|     US|     N126UW|        1|             1|
|     MQ|     N532MQ|        1|            45|
|     EV|     N14153|        1|             5|
|     EV|     N837AS|        1|            15|
|     OO|     N945SW|        1|             9|
|     OO|     N809CA|        1|             9|
|     UA|     N38727|        1|             1|
|     B6|     N935JB|        1|             4|
|     AA|     N3LXAA|        1|             8|
|     AS|    

In [331]:
#SPARK SQL
spark.sql('SELECT AIRLINE, TAIL_NUMBER, sum(CANCELLED) AS Total FROM flights WHERE CANCELLED = 1 GROUP BY AIRLINE, TAIL_NUMBER ORDER BY Total DESC').show(30)

+-------+-----------+-----+
|AIRLINE|TAIL_NUMBER|Total|
+-------+-----------+-----+
|     US|       null| 3068|
|     UA|       null| 2346|
|     WN|       null| 1023|
|     AA|       null|  961|
|     F9|       null|  224|
|     MQ|     N655MQ|   79|
|     MQ|     N696MQ|   68|
|     MQ|     N610MQ|   67|
|     MQ|     N630MQ|   66|
|     MQ|     N909MQ|   61|
|     EV|     N902EV|   61|
|     MQ|     N544MQ|   60|
|     MQ|     N855MQ|   60|
|     MQ|     N660MQ|   59|
|     MQ|     N820MQ|   58|
|     MQ|     N851MQ|   57|
|     MQ|     N602MQ|   57|
|     MQ|     N608MQ|   57|
|     MQ|     N9EAMQ|   57|
|     MQ|     N645MQ|   57|
|     MQ|     N609MQ|   57|
|     MQ|     N618MQ|   56|
|     MQ|     N908MQ|   56|
|     MQ|     N527MQ|   56|
|     MQ|     N694MQ|   55|
|     MQ|     N636MQ|   55|
|     MQ|     N676MQ|   55|
|     MQ|     N617MQ|   55|
|     MQ|     N513MQ|   55|
|     MQ|     N531MQ|   55|
+-------+-----------+-----+
only showing top 30 rows



### 8- Which origin airport has experienced arrival and departure delays more often?

In [252]:
#DataFrame API
df_originarrival = df.groupBy('ORIGIN_AIRPORT').avg('ARRIVAL_DELAY').show()

+--------------+-------------------+
|ORIGIN_AIRPORT| avg(ARRIVAL_DELAY)|
+--------------+-------------------+
|           BGM|  18.36111111111111|
|           PSE| 4.0198675496688745|
|           INL| 12.268292682926829|
|           MSY|  2.171060852180432|
|           PPG| 16.761904761904763|
|           GEG| 0.6990788126919141|
|           SNA| 2.9120745542949757|
|           BUR|  4.235111341273951|
|           GRB|  8.636865342163356|
|           GTF| 1.2852852852852852|
|           IDA|-1.3317191283292977|
|           GRR|  7.639982502187227|
|           JLN| 16.126984126984127|
|           EUG| 3.6726272352132048|
|           PSG|  3.870503597122302|
|           GSO| 10.132481506388702|
|           PVD| 4.4658264663111975|
|           MYR| 11.541666666666666|
|           OAK|  5.539095647957328|
|           MSN|  8.401184706515886|
+--------------+-------------------+
only showing top 20 rows



In [253]:
#SPARK SQL
df_originarrival = spark.sql('SELECT ORIGIN_AIRPORT, round(avg(ARRIVAL_DELAY), 2) AS Average_Arrival_Delay FROM flights GROUP BY ORIGIN_AIRPORT ORDER BY Average_Arrival_Delay')
df_originarrival.show()

+--------------+---------------------+
|ORIGIN_AIRPORT|Average_Arrival_Delay|
+--------------+---------------------+
|           CNY|               -22.19|
|           BTM|               -13.95|
|           YAK|               -11.33|
|           LWS|               -10.76|
|           HLN|                -8.12|
|           TWF|                -7.79|
|           ADQ|                -6.77|
|           CDV|                -6.74|
|           MSO|                -6.45|
|           FCA|                 -5.4|
|           GCC|                 -5.3|
|           BJI|                -5.22|
|           PSC|                -3.94|
|           BIL|                -3.28|
|           ITH|                 -3.0|
|           GUM|                -2.94|
|           VEL|                -2.55|
|           ANC|                 -2.5|
|           CPR|                -2.05|
|           EKO|                -1.77|
+--------------+---------------------+
only showing top 20 rows



In [254]:
#DataFrame API
df_origindep = df.groupBy('DESTINATION_AIRPORT').avg('DEPARTURE_DELAY').show()

+-------------------+--------------------+
|DESTINATION_AIRPORT|avg(DEPARTURE_DELAY)|
+-------------------+--------------------+
|                BGM|   21.97222222222222|
|                INL|   3.292682926829268|
|                PSE|   9.175675675675675|
|                MSY|   9.555333998005983|
|                PPG|   2.761904761904762|
|                GEG|   8.660860655737705|
|                SNA|   6.790483914571506|
|                BUR|   8.258714175058094|
|                GRB|   15.81687014428413|
|                GTF|   6.371601208459214|
|                IDA|   9.491525423728813|
|                GRR|  13.355343179843615|
|                JLN|            12.65625|
|                EUG|   10.91655266757866|
|                PSG|  12.838028169014084|
|                MYR|  10.369843527738265|
|                GSO|  14.885906040268456|
|                PVD|    11.9995183044316|
|                OAK|   9.843446601941748|
|                MSN|  12.604177825388323|
+----------

In [255]:
#SPARK SQL
df_origindeparture = spark.sql('SELECT DESTINATION_AIRPORT, round(avg(DEPARTURE_DELAY), 2) AS Average_Departure_Delay FROM flights GROUP BY DESTINATION_AIRPORT ORDER BY Average_Departure_Delay')
df_origindeparture.show()

+-------------------+-----------------------+
|DESTINATION_AIRPORT|Average_Departure_Delay|
+-------------------+-----------------------+
|                VEL|                  -5.13|
|                EKO|                  -4.72|
|                BTM|                  -4.19|
|                LWS|                  -3.33|
|                CNY|                  -3.02|
|                PIH|                  -2.83|
|                CDC|                  -2.69|
|                TWF|                  -2.18|
|                YAK|                  -1.89|
|                ADK|                   -0.4|
|                CDV|                   -0.2|
|                ADQ|                   0.32|
|                BLI|                    1.6|
|                LIH|                   1.68|
|                SGU|                   1.86|
|                ITO|                   1.88|
|                BJI|                   2.22|
|                BET|                   2.54|
|                YUM|             

CNY Airport often experienced arrival delay while VEL airport often experienced departure delay.

### 9- Which destination took the longest flight?

In [284]:
df = df.withColumn('AirTime_Hrs', df.AIR_TIME * (1/60))

In [285]:
df.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY',
 'AirTime_Hrs']

In [258]:
df.show(5)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|       AirTime_Hrs|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+

In [259]:
#DataFrame API
df_hrs = df.groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DISTANCE').avg('AirTime_Hrs').show()

+--------------+-------------------+--------+------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|  avg(AirTime_Hrs)|
+--------------+-------------------+--------+------------------+
|           BZN|                DEN|     524|1.1730842911877395|
|           MGM|                DFW|     622|1.7696396396396397|
|           PDX|                ONT|     838|1.8424549549549547|
|           SNA|                PHX|     338|0.8841523341523343|
|           MCO|                LAS|    2039| 4.777083333333333|
|           MIA|                DTW|    1145| 2.615714285714285|
|           BWI|                CHS|     472|1.2835964912280706|
|           EWR|                SAN|    2425| 5.537037037037037|
|           CMH|                DEN|    1154|2.9097849462365577|
|           DEN|                MEM|     872|1.8197399527186764|
|           ECP|                ATL|     240|0.7115869017632247|
|           SAN|                MCO|    2149|             4.125|
|           DEN|         

In [260]:
#to create new view with new column
df.createOrReplaceTempView('flights')

In [268]:
df_hrs = spark.sql('SELECT * FROM flights').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+------------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|       AirTime_Hrs|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+

In [269]:
#SPARK SQL
df_hrs = spark.sql('SELECT ORIGIN_AIRPORT, DESTINATION_AIRPORT, DISTANCE, round(avg(AirTime_Hrs), 2)  AS Hours FROM flights GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT, DISTANCE ORDER BY Hours DESC').show()

+--------------+-------------------+--------+-----+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|Hours|
+--------------+-------------------+--------+-----+
|           JFK|                HNL|    4983|10.82|
|           EWR|                HNL|    4962|10.55|
|           IAD|                HNL|    4817| 10.2|
|           ATL|                HNL|    4502| 9.58|
|           HNL|                JFK|    4983| 8.91|
|           HNL|                EWR|    4962| 8.83|
|           ORD|                HNL|    4243| 8.75|
|           ORD|                OGG|    4184|  8.7|
|           HNL|                IAD|    4817| 8.48|
|           DFW|                HNL|    3784| 8.19|
|           IAH|                HNL|    3904| 8.09|
|           DFW|                OGG|    3711| 8.02|
|           HNL|                ATL|    4502| 7.96|
|           HNL|                ORD|    4243| 7.44|
|           HNL|                GUM|    3801| 7.42|
|           DEN|                LIH|    3414| 7.39|
|           

### 10- What was the average speed (km/h) for each flight?

In [303]:
df_speed = df.withColumn('Speed', df.DISTANCE/df.AIR_TIME)

In [304]:
df_speed.show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+-----------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|            Speed|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--

In [305]:
df_speed.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY',
 'Speed']

In [312]:
df_speed.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [307]:
#DataFrame API
df_avgspeed = df_speed.groupBy('TAIL_NUMBER', 'Speed').avg('Speed').show()

+-----------+-----------------+-----------------+
|TAIL_NUMBER|            Speed|       avg(Speed)|
+-----------+-----------------+-----------------+
|     N3JYAA|7.258278145695364|7.258278145695364|
|     N477HA|5.666666666666667|5.666666666666667|
|     N57864| 7.71608832807571| 7.71608832807571|
|     N508AS|7.376470588235295|7.376470588235295|
|     N835UA|6.548387096774194|6.548387096774194|
|     N816SK|7.098360655737705|7.098360655737705|
|     N523AS|7.100609756097561|7.100609756097561|
|     N526NK|5.051282051282051|5.051282051282051|
|     N452WN|         6.203125|         6.203125|
|     N8616C|7.785714285714286|7.785714285714286|
|     N8311Q|6.543478260869565|6.543478260869565|
|     N449SW|5.953846153846154|5.953846153846154|
|     N17115|8.904347826086957|8.904347826086957|
|     N625SW|7.283464566929134|7.283464566929134|
|     N203FR|8.146788990825687|8.146788990825687|
|     N759GS|              5.0|              5.0|
|     N494CA|6.632911392405063|6.632911392405063|


In [318]:
#to create temp. view
df_speed.createOrReplaceTempView('speeds')

In [322]:
spark.sql('SELECT TAIL_NUMBER, round(avg(Speed), 2) AS Average_Speed FROM speeds GROUP BY TAIL_NUMBER ORDER BY Average_Speed DESC').show()

+-----------+-------------+
|TAIL_NUMBER|Average_Speed|
+-----------+-------------+
|     N78001|         9.52|
|     N222UA|          9.3|
|     N845MH|         9.27|
|     N177UA|         9.23|
|     N777UA|         9.07|
|     N7ADAA|         9.06|
|     N7ATAA|         9.04|
|     N173DZ|         9.03|
|     N116UA|         9.02|
|     N7ANAA|          9.0|
|     N812NW|         8.82|
|     N818NW|         8.82|
|     N74007|         8.81|
|     N7BCAA|          8.8|
|     N772UA|         8.79|
|     N214UA|         8.77|
|     N69020|         8.76|
|     N768UA|         8.76|
|     N213UA|         8.75|
|     N211UA|         8.75|
+-----------+-------------+
only showing top 20 rows

